Roll procedure

1. Update df_elo using df elo updater notebook
2. Update merged_gw using the github data, literally just download the new version and replace the current one in documents folder
3. Update event=GW in json_fixtures

Then it should work


Should start looking back more than 3 weeks once we have more data for this season

In [1]:
import requests
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
import scipy
from io import StringIO
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
warnings.filterwarnings("ignore")

In [42]:
url = "https://github.com/vaastav/Fantasy-Premier-League/blob/master/data/2023-24/gws/merged_gw.csv?raw=true"
response = requests.get(url)

csv_data = StringIO(response.text)
merged_gw_df = pd.read_csv(csv_data)

In [2]:
def get_date(datetime):
    return datetime.date()

def pos_to_number(pos):
    if pos=='GK':
        return 0
    elif pos=='DEF':
        return 1
    elif pos=='MID':
        return 2
    elif pos=='FWD':
        return 3

In [3]:
merged_20_21 = pd.read_csv('merged_gw_20_21.csv')
merged_21_22 = pd.read_csv('merged_gw_21_22.csv')[merged_20_21.columns]
merged_22_23 = pd.read_csv('merged_gw_22_23.csv')[merged_20_21.columns]
merged_23_24 = merged_gw_df[merged_20_21.columns]
teams_20_21 = pd.read_csv('teams_20_21.csv')
teams_21_22 = pd.read_csv('teams_21_22.csv')
teams_22_23 = pd.read_csv('teams_22_23.csv')
teams_23_24 = pd.read_csv('teams_23_24.csv')

merged_20_21['opponent_team_name'] = merged_20_21['opponent_team'].map(teams_20_21[['name', 'id']].set_index('id').to_dict()['name'])
merged_21_22['opponent_team_name'] = merged_21_22['opponent_team'].map(teams_21_22[['name', 'id']].set_index('id').to_dict()['name'])
merged_22_23['opponent_team_name'] = merged_22_23['opponent_team'].map(teams_22_23[['name', 'id']].set_index('id').to_dict()['name'])
merged_23_24['opponent_team_name'] = merged_23_24['opponent_team'].map(teams_23_24[['name', 'id']].set_index('id').to_dict()['name'])

merged_20_21['season'] = '20-21'
merged_21_22['season'] = '21-22'
merged_22_23['season'] = '22-23'
merged_23_24['season'] = '23-24'

merged_list = [merged_20_21, merged_21_22, merged_22_23, merged_23_24]

merged_df = pd.concat(merged_list)

merged_df['is_this_week'] = False

In [4]:
df_elo = pd.read_csv('rankings_2020_2023.csv').drop(columns='Unnamed: 0')

In [5]:
merged_df['date'] = pd.to_datetime(merged_df['kickoff_time']).apply(get_date).apply(str)

In [6]:
merged_df = merged_df.merge(df_elo, how='left', on=['team', 'date'])
merged_df = merged_df.rename(columns={'elo': 'team_elo'})
merged_df = merged_df.merge(df_elo, how='left', left_on=['opponent_team_name', 'date'], right_on=['team', 'date'])
merged_df = merged_df.rename(columns={'elo': 'opponent_elo'})

merged_df = merged_df.drop(columns='team_y')

In [7]:
merged_df['team_score'] = merged_df['team_h_score']*merged_df['was_home'] + merged_df['team_a_score']*(1-merged_df['was_home'])
merged_df['opponent_score'] = merged_df['team_a_score']*merged_df['was_home'] + merged_df['team_h_score']*(1-merged_df['was_home'])
merged_df['goal_diff'] = merged_df['team_score'] - merged_df['opponent_score']
merged_df['elo_diff'] = merged_df['team_elo'] - merged_df['opponent_elo']

In [8]:
original_cols = merged_df.columns

In [9]:
known_cols = ['name', 'date', 'total_points', 'position', 'transfers_balance', 'transfers_in', 'transfers_out', 'value', 'was_home', 'team_elo', 'opponent_elo']

useful_cols = ['xP', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'goals_conceded', 'goals_scored', 'ict_index', 'influence', 'minutes', 'own_goals', 'penalties_missed', 'red_cards', 'saves', 'team_score', 'opponent_score', 'goal_diff', 'elo_diff', 'threat', 'total_points', 'transfers_balance', 'value', 'was_home', 'yellow_cards', 'team_elo', 'opponent_elo']

In [10]:
json_bootstrap = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').json()
json_fixtures = requests.get('https://fantasy.premierleague.com/api/fixtures?event=6').json()
df_fixtures = pd.DataFrame(json_fixtures)
df_bootstrap = pd.DataFrame(json_bootstrap['elements'])
team_map = pd.read_csv('teams.csv')[['id', 'name']].set_index('id').to_dict()['name']

fixtures_sorted_1 = df_fixtures[['team_a', 'team_h']].set_index('team_a')
fixtures_sorted_1 = fixtures_sorted_1.rename(columns={'team_h':'opponent'})
fixtures_sorted_1['was_home'] = False
fixtures_sorted_2 = df_fixtures[['team_a', 'team_h']].set_index('team_h')
fixtures_sorted_2 = fixtures_sorted_2.rename(columns={'team_a':'opponent'})
fixtures_sorted_2['was_home'] = True
fixtures_sorted = pd.concat([fixtures_sorted_1, fixtures_sorted_2])

team_map_elo = {
    'Man United': 'Man Utd',
    'Tottenham': 'Spurs',
    "Forest": "Nott'm Forest",
    'Sheffield United': 'Sheffield Utd'
}


yesterday_date = dt.date.today()-dt.timedelta(days=1)
elo_df = pd.read_csv('http://api.clubelo.com/'+str(yesterday_date))
elo_df = elo_df[elo_df['Country'].eq('ENG')]
elo_df['Club'] = elo_df['Club'].map(team_map_elo).fillna(elo_df['Club'])
elo_df = elo_df.set_index('Club')[['Elo']]
elo_dict = elo_df.to_dict()['Elo']

df_bootstrap['name'] = df_bootstrap['first_name']+' '+df_bootstrap['second_name']
df_bootstrap['position'] = df_bootstrap['element_type']-1
df_bootstrap['selected_by_percent'] = df_bootstrap['selected_by_percent'].apply(float)
df_bootstrap['value'] = df_bootstrap['now_cost']
df_bootstrap['team_id'] = df_bootstrap['team']
df_bootstrap['team'] = df_bootstrap['team'].map(team_map)
df_bootstrap = df_bootstrap.merge(fixtures_sorted, how='left', left_on='team_id', right_index=True)
df_bootstrap['opponent_team_name'] = df_bootstrap['opponent'].map(team_map)
df_bootstrap['team_elo'] = df_bootstrap['team'].map(elo_dict)
df_bootstrap['opponent_elo'] = df_bootstrap['opponent_team_name'].map(elo_dict)
df_bootstrap['elo_diff'] = df_bootstrap['team_elo'] - df_bootstrap['opponent_elo']
df_bootstrap['transfers_balance'] = df_bootstrap['transfers_in'] - df_bootstrap['transfers_out']
df_bootstrap['date'] = str(dt.date.today())

for col in useful_cols:
    if col not in df_bootstrap.columns:
        df_bootstrap[col] = 0
        
df_bootstrap['is_this_week'] = True

In [11]:
merged_df = pd.concat([merged_df, df_bootstrap])

In [12]:
player_list = list(merged_df['name'].unique())

In [13]:
df_list = []
cols_to_use = []
for player in player_list:
    temp_df = merged_df[merged_df['name'].eq(player)].sort_values('date')
    for col in useful_cols:
        col1 = str(col)+'-1'
        col2 = str(col)+'-2'
        col3 = str(col)+'-3'
        col4 = str(col)+'-4'
        temp_df[col1] = temp_df[col].shift(1)
        temp_df[col2] = temp_df[col].shift(2)
        temp_df[col3] = temp_df[col].shift(3)
        temp_df[col4] = temp_df[col].shift(4)
    df_list.append(temp_df)
    
for col in useful_cols:
    col1 = str(col)+'-1'
    col2 = str(col)+'-2'
    col3 = str(col)+'-3'
    col4 = str(col)+'-4'
    cols_to_use.append(col1)
    cols_to_use.append(col2)
    cols_to_use.append(col3)
    cols_to_use.append(col4)
    
merged_df = pd.concat(df_list)

In [14]:
this_week_df = merged_df[merged_df['is_this_week'].eq(True)].dropna(axis=1, how='all')[known_cols+cols_to_use]
merged_df = merged_df[merged_df['is_this_week'].eq(False)].dropna(axis=1, how='all').dropna()

In [15]:
merged_df = merged_df.dropna()
merged_df['position'] = merged_df['position'].apply(pos_to_number)
merged_df_slim = merged_df[known_cols+cols_to_use]

merged_df_slim = merged_df_slim.sort_values('date')

In [16]:
cols_to_conv = ['creativity-1','creativity-2','creativity-3','creativity-4', 'ict_index-1', 'ict_index-2', 'ict_index-3', 'ict_index-4', 'influence-1', 'influence-2', 'influence-3', 'influence-4', 'threat-1', 'threat-2', 'threat-3', 'threat-4', 'was_home', 'was_home-1', 'was_home-2', 'was_home-3', 'was_home-4']
for col in cols_to_conv:
    merged_df_slim[col] = merged_df_slim[col].astype(float)

In [17]:
df = merged_df_slim.select_dtypes(include='number').dropna()
#df = df[df['position'].eq(3)]

In [18]:
X = df.drop(columns=['total_points'])
y = df['total_points']
train_size = 0.8
train_index = int(train_size * len(df))
X_train, X_test = X[:train_index], X[train_index:]
y_train, y_test = y[:train_index], y[train_index:]

rf_model = RandomForestRegressor(n_estimators=1000, max_features = 5)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R2 Score:", r2)

Mean Squared Error: 3.704797683428339
R2 Score: 0.3124975170747152


In [19]:
this_week_df[this_week_df['name'].eq('Erling Haaland')]

,name,date,total_points,position,transfers_balance,transfers_in,transfers_out,value,was_home,team_elo,...,yellow_cards-3,yellow_cards-4,team_elo-1,team_elo-2,team_elo-3,team_elo-4,opponent_elo-1,opponent_elo-2,opponent_elo-3,opponent_elo-4
440,Erling Haaland,2023-09-22,45,3,342716,446099,103383,141,True,2092.54834,...,0.0,0.0,2087.354004,2084.956787,2083.702148,2080.850342,1807.522583,1738.858398,1631.764282,1887.064697


In [20]:
X_this_week = this_week_df.dropna().drop(columns=['total_points', 'name', 'date'])
pred_this_week = rf_model.predict(X_this_week)

In [21]:
X_this_week['predicted_points'] = pred_this_week

In [22]:
this_week_df_pred = this_week_df.loc[X_this_week.index]
this_week_df_pred['predicted_points'] = pred_this_week

In [23]:
this_week_df_pred['point_value'] = this_week_df_pred['predicted_points']/this_week_df_pred['value']

In [37]:
this_week_df_pred[this_week_df_pred['position'].eq(3)][['name', 'position', 'predicted_points', 'value', 'point_value']].sort_values('point_value', ascending=False).head(10)

,name,position,predicted_points,value,point_value
429,Julián Álvarez,3,5.436,69,0.078783
148,João Pedro Junqueira de Jesus,3,3.948,54,0.073111
586,Cameron Archer,3,3.142,45,0.069822
268,Odsonne Edouard,3,3.889,56,0.069446
129,Yoane Wissa,3,4.148,61,0.068000
342,Carlos Vinícius Alves Morais,3,3.211,48,0.066896
184,Lyle Foster,3,3.257,50,0.065140
145,Evan Ferguson,3,3.835,60,0.063917
656,Michail Antonio,3,3.484,60,0.058067
237,Nicolas Jackson,3,4.045,70,0.057786


In [39]:
this_week_df_pred[this_week_df_pred['position'].eq(2)][['name', 'position', 'predicted_points', 'value', 'point_value']].sort_values('predicted_points', ascending=False).head(10)

,name,position,predicted_points,value,point_value
374,Mohamed Salah,2,5.169,125,0.041352
448,Rodrigo Hernandez,2,5.145,57,0.090263
242,Raheem Sterling,2,5.068,72,0.070389
438,Phil Foden,2,5.055,76,0.066513
633,James Maddison,2,4.864,79,0.061570
430,Bernardo Veiga de Carvalho e Silva,2,4.820,64,0.075313
17,Bukayo Saka,2,4.728,87,0.054345
119,Bryan Mbeumo,2,4.680,69,0.067826
156,Kaoru Mitoma,2,4.485,65,0.069000
680,James Ward-Prowse,2,4.447,62,0.071726


In [31]:
json_team = requests.get('https://fantasy.premierleague.com/api/entry/3286047/event/5/picks/').json()['picks']
df_team = pd.DataFrame(json_team)
id_list = pd.read_csv('player_idlist.csv')
id_list['name'] = id_list['first_name'] + ' ' + id_list['second_name']
id_list = id_list.set_index('id')[['name']].to_dict()['name']
df_team['name'] = df_team['element'].map(id_list)

In [32]:
chosen_players = list(df_team['name'].unique())

In [33]:
my_team = this_week_df_pred[this_week_df_pred['name'].isin(chosen_players)][['name', 'position', 'predicted_points', 'value', 'point_value']]

In [34]:
my_team.sort_values('predicted_points')

,name,position,predicted_points,value,point_value
46,Ezri Konsa Ngoyo,1,2.510,45,0.055778
123,Ethan Pinnock,1,3.197,45,0.071044
499,Sven Botman,1,3.308,46,0.071913
308,Jordan Pickford,0,3.366,44,0.076500
144,Pervis Estupiñán,1,3.431,53,0.064736
12,Martin Ødegaard,2,3.463,85,0.040741
57,Ollie Watkins,3,3.634,79,0.046000
268,Odsonne Edouard,3,3.889,56,0.069446
460,Bruno Borges Fernandes,2,4.054,84,0.048262
330,Bernd Leno,0,4.350,46,0.094565


In [811]:
actual_shown = list(this_week_df_pred[this_week_df_pred['name'].isin(chosen_players)]['name'].unique())
[x for x in chosen_players if x not in actual_shown]

[]

In [244]:
importance_scores = rf_model.feature_importances_

feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importance_scores})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

In [ ]:
Mean Squared Error: 3.603470236656732
R2 Score: 0.31593802108182123

Model 1 overall score:

Mean Squared Error: 3.5952346517704847
R2 Score: 0.31810399879897255

GK score: 

Mean Squared Error: 2.587631659627329
R2 Score: 0.42445418223008935

DEF score: 

Mean Squared Error: 3.744577954420432
R2 Score: 0.24137439609934275

MID score:

Mean Squared Error: 3.5478586540726207
R2 Score: 0.33009476371421087

FWD score:

Mean Squared Error: 4.6807401643167035
R2 Score: 0.33249371469894606